# Train 

In [1]:
from time import time
from tqdm import tqdm
import torch
import torch.nn
import torch.optim
import numpy as np


In [2]:
import model

In [3]:
import data


e:\Luciano\Salt\env\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
it=iter(data.train_loader)
real_batch = next(iter(it))


In [5]:
cinn = model.MNIST_cINN(5e-4)
cinn.cuda()
scheduler = torch.optim.lr_scheduler.MultiStepLR(cinn.optimizer, milestones=[20, 40], gamma=0.1)

N_epochs = 60
t_start = time()
nll_mean = []

In [6]:
print('Epoch\tBatch/Total \tTime \tNLL train\tNLL val\tLR')
for epoch in range(N_epochs):
    for i, (x, l) in enumerate(data.train_loader): #data.train_loader):
        x, l = x.cuda(), l.cuda()
        z, log_j = cinn(x, l)

        nll = torch.mean(z**2) / 2 - torch.mean(log_j) / model.ndim_total
        nll.backward()
        torch.nn.utils.clip_grad_norm_(cinn.trainable_parameters, 10.)
        nll_mean.append(nll.item())
        cinn.optimizer.step()
        cinn.optimizer.zero_grad()

        if not i % 50:
            with torch.no_grad():
                z, log_j = cinn(data.val_x, data.val_l) #cinn(data.val_x, data.val_l)
                nll_val = torch.mean(z**2) / 2 - torch.mean(log_j) / model.ndim_total

            print('%.3i \t%.5i/%.5i \t%.2f \t%.6f\t%.6f\t%.2e' % (epoch,
                                                            i, len(data.train_loader),
                                                            (time() - t_start)/60.,
                                                            np.mean(nll_mean),
                                                            nll_val.item(),
                                                            cinn.optimizer.param_groups[0]['lr'],
                                                            ), flush=True)
            nll_mean = []
    scheduler.step()

Epoch	Batch/Total 	Time 	NLL train	NLL val	LR
000 	00000/00230 	0.11 	0.538928	0.176847	5.00e-04
000 	00050/00230 	0.19 	-0.666140	-1.340032	5.00e-04
000 	00100/00230 	0.27 	-1.241935	-1.522886	5.00e-04
000 	00150/00230 	0.35 	-1.317406	-1.620022	5.00e-04
000 	00200/00230 	0.42 	-1.351816	-1.631858	5.00e-04
001 	00000/00230 	0.53 	-1.364375	-1.669222	5.00e-04
001 	00050/00230 	0.61 	-1.387519	-1.677190	5.00e-04
001 	00100/00230 	0.68 	-1.398561	-1.703947	5.00e-04
001 	00150/00230 	0.75 	-1.411586	-1.711710	5.00e-04
001 	00200/00230 	0.83 	-1.419027	-1.744163	5.00e-04
002 	00000/00230 	0.93 	-1.423610	-1.752489	5.00e-04
002 	00050/00230 	1.01 	-1.437380	-1.748425	5.00e-04
002 	00100/00230 	1.08 	-1.446082	-1.767792	5.00e-04
002 	00150/00230 	1.16 	-1.451494	-1.781033	5.00e-04
002 	00200/00230 	1.23 	-1.457667	-1.790701	5.00e-04
003 	00000/00230 	1.33 	-1.461937	-1.790992	5.00e-04
003 	00050/00230 	1.41 	-1.473144	-1.803693	5.00e-04
003 	00100/00230 	1.48 	-1.477325	-1.806622	5.00e-04
00

In [7]:
torch.save(cinn.state_dict(), 'output/mnist_cinn.pt')